In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 9 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

19 December 2022 ; 16:28:18


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,12,0,5.8,1.1,2.0,0.542,...,4.17,113.54,109.23,4.31,3.78,113.43,109.5,3.93,1,1
1,Bobby Portis,PF,27,MIL,29,8,26.5,6.1,12.1,0.504,...,4.17,113.54,109.23,4.31,3.78,113.43,109.5,3.93,0,1
3,George Hill,PG,36,MIL,29,0,19.8,1.6,3.8,0.427,...,4.17,113.54,109.23,4.31,3.78,113.43,109.5,3.93,0,1
7,Jordan Nwora,SF,24,MIL,25,3,17.1,2.1,5.7,0.364,...,4.17,113.54,109.23,4.31,3.78,113.43,109.5,3.93,0,1
10,MarJon Beauchamp,SF,22,MIL,18,6,15.5,2.2,5.6,0.400,...,4.17,113.54,109.23,4.31,3.78,113.43,109.5,3.93,1,1


In [5]:
# Set training data
train_data = train_data[train_data['Year'] < 2020] # DELETE THIS LINE
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Malcolm Brogdon,23.8,26,0,BOS,18.7,2.207692,5.992308,13.8,2,3.0,4.100000,0.335961
1,Bobby Portis,26.5,29,8,MIL,19.2,1.979310,7.351724,14.4,1,2.0,3.675862,0.269015
2,Bones Hyland,21.1,21,0,DEN,16.2,0.780952,2.342857,13.8,2,2.1,1.171429,0.116541
3,Tyus Jones,24.0,29,5,MEM,15.9,1.696552,5.372414,10.9,1,1.6,3.675862,0.060491
4,Christian Wood,27.0,27,2,DAL,20.1,1.822222,5.770370,17.5,9,1.7,2.733333,0.058382


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)